In [10]:
import os
from pathlib import Path

In [11]:
profile_dir = Path("./profile")
rhasspy_dir = Path("/usr/lib/rhasspy")

In [12]:
os.putenv("profile_dir", str(profile_dir))
os.putenv("rhasspy_dir", str(rhasspy_dir))

In [14]:
!cat ./profile/sentences.ini

[GetTime]
what time is it
tell me the time

[GetTemperature]
whats the temperature
how (hot | cold) is it

[GetGarageState]
is the garage door (open | closed)

[ChangeLightState]
light_name = ((living room lamp | garage light) {name}) | <ChangeLightColor.light_name>
light_state = (on | off) {state}

turn <light_state> [the] <light_name>
turn [the] <light_name> <light_state>

[ChangeLightColor]
light_name = (bedroom light) {name}
color = (red | green | blue) {color}

set [the] <light_name> [to] <color>
make [the] <light_name> <color>

In [15]:
!cat ./profile/profile.yml

speech-to-text:
  pocketsphinx:
    acoustic-model: !env "${rhasspy_dir}/languages/english/en-us_pocketsphinx-cmu/acoustic_model"
    language-model: !env "${profile_dir}/language_model.txt"
    dictionary: !env "${profile_dir}/dictionary.txt"

intent-recognition:
  fsticuffs:
    intent-fst: !env "${profile_dir}/intent.fst"
    skip-unknown: true
    fuzzy: true

training:
  sentences-file: !env "${profile_dir}/sentences.ini"
  intent-fst: !env "${profile_dir}/intent.fst"
  language-model: !env "${profile_dir}/language_model.txt"
  dictionary: !env "${profile_dir}/dictionary.txt"
  base-dictionary: !env "${rhasspy_dir}/languages/english/en-us_pocketsphinx-cmu/base_dictionary.txt"
  grapheme-to-phoneme-model: !env "${rhasspy_dir}/languages/english/en-us_pocketsphinx-cmu/g2p.fst"


In [18]:
!rhasspy-train --profile ./profile

DEBUG:rhasspy_train:Profile at /home/hansenm/opt/rhasspy-services/docs/notebooks/profile
-- grammars
-- grammar_fsts:GetGarageState_fst
-- grammar_fsts:GetTemperature_fst
-- grammar_fsts:ChangeLightColor_fst
-- grammar_fsts:GetTime_fst
-- grammar_fsts:ChangeLightState_fst
.  grammar_fsts:slot_fsts
-- intent_fst
-- language_model:intent_counts
-- language_model:intent_model
-- language_model:intent_arpa
-- vocab
-- vocab_dict


In [31]:
!rhasspy-stream-uri ./wav/turn_on_living_room_lamp.wav | \
  rhasspy-pocketsphinx-profile --profile ./profile | \
  jq .  

acoustic_model='/usr/lib/rhasspy/languages/english/en-us_pocketsphinx-cmu/acoustic_model'
language_model='/home/hansenm/opt/rhasspy-services/docs/notebooks/profile/language_model.txt'
dictionary='/home/hansenm/opt/rhasspy-services/docs/notebooks/profile/dictionary.txt'
{
  "text": "turn on the living room lamp",
  "transcribe_seconds": 0.11511564254760742,
  "likelihood": 0.6240313577054549
}


In [32]:
!rhasspy-stream-uri ./wav/turn_on_living_room_lamp.wav | \
  rhasspy-pocketsphinx-profile --profile ./profile | \
  rhasspy-fsticuffs-profile --profile ./profile | \
  jq .

intent_fst='/home/hansenm/opt/rhasspy-services/docs/notebooks/profile/intent.fst'
skip_unknown=''
fuzzy='True'
acoustic_model='/usr/lib/rhasspy/languages/english/en-us_pocketsphinx-cmu/acoustic_model'
language_model='/home/hansenm/opt/rhasspy-services/docs/notebooks/profile/language_model.txt'
dictionary='/home/hansenm/opt/rhasspy-services/docs/notebooks/profile/dictionary.txt'
{
  "text": "turn on the living room lamp",
  "intent": {
    "name": "ChangeLightState",
    "confidence": 1
  },
  "entities": [
    {
      "entity": "state",
      "value": "on",
      "raw_value": "on",
      "start": 5,
      "end": 7
    },
    {
      "entity": "name",
      "value": "living room lamp",
      "raw_value": "living room lamp",
      "start": 12,
      "end": 28
    }
  ],
  "raw_text": "turn on the living room lamp",
  "tokens": [
    "turn",
    "on",
    "the",
    "living",
    "room",
    "lamp"
  ],
  "raw_tokens": [
    "turn",
    "on",
    "the",
    "living",
    "room",
    "lamp